# Game Suggestion System

In this project, I have implemented a game suggestion system based on Steam data obtained from Kaggle.com. The steps I took to complete this work were as follows:

- **Import Libraries:** Imported all the necessary libraries.
- **Database Management:** Used `sqlite3` to manage the data with SQL for faster performance.
- **Data Transformation:** Transformed the data to make it optimal for the machine learning algorithm.
- **Nearest Neighbors Algorithm:** Utilized the Nearest Neighbors algorithm to find similarities in the data.
- **Recommendations:** Generated and presented some recommendations.

I began by importing all the required libraries.

In [63]:
import sqlite3
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
import re 


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MinMaxScaler


To enhance query performance, two indexes have been created.
These indexes facilitate quicker retrieval of data, improving the overall efficiency.

In [64]:
conn = sqlite3.connect('GameRecomendations_on_Steam.db')
cursor = conn.cursor()

cursor.execute('CREATE INDEX IF NOT EXISTS idx_Recommendations_user_id ON Recommendations (user_id);')
cursor.execute('CREATE INDEX IF NOT EXISTS idx_Recommendations_app_id ON Recommendations (app_id);')

conn.commit()
conn.close()

Then, I used SQL queries to select, join, and filter the data. In this step, I obtained:

- The games that the user recommends. I will use this information to recommend more games to the user.
- All Steam games until 2019. This data will be used to train the algorithm.

In [65]:
# Others interesting user_id: 34580 504588
with sqlite3.connect("GameRecomendations_on_Steam.db") as conn:
    query = '''
        SELECT s.name as title
        FROM Recommendations r
        INNER JOIN Steam s 
        ON r.app_id = s.appid
        WHERE r.user_id = 45805 and is_recommended = "true";
    '''
    user = pd.read_sql_query(query, conn)
    
    query = '''
        SELECT s.name as title, s.genres, s.price, s.steamspy_tags
        FROM Games AS g 
        INNER JOIN Steam s
        ON g.app_id = s.appid
        ORDER BY app_id
    ''' 
    games_df = pd.read_sql_query(query, conn)


The columns 'categories', 'genres', and 'steamspy_tags' contain multiple attributes separated by semicolons. To address this, I split these attributes into different columns. Subsequently, I use One Hot Encoding (OHE) to convert the categorical data into a format suitable for machine learning algorithms. Afterward, I remove the duplicated columns generated by OHE.

In [66]:
def OHERemoveDuplicated(df, column):
    column_split = df[column].str.split(';', expand=True)
    df_drop = df.drop([column], axis =1 )

    column_encoded = pd.get_dummies(column_split, prefix=column)
    column_encoded = column_encoded.astype(int)

    combinated_columns = column_encoded.copy()
    l = []
    for row in column_encoded.loc[:, column_encoded.columns.duplicated()].columns:
        if row in l:
            continue
        combinated_columns[row] = combinated_columns[row].sum(axis=1)
        column_encoded = column_encoded.drop(row, axis=1)
        l.append(row)

    df_concat = pd.concat([df_drop, combinated_columns, column_encoded], axis=1)
    return df_concat

Then, I apply the funtion on the data frame

In [67]:

X = OHERemoveDuplicated(games_df, "genres" )
X = OHERemoveDuplicated(X, "steamspy_tags" )
c = X.columns

In the following code, I define custom transformers:

- **TextPreprocessor:** This transformer removes numbers and punctuation marks from text, preparing the data for the next transformer.
- **TfidfSumVectorizer:** This transformer is used to process categorical data, specifically titles and genres. It assigns a score to each word based on its frequency in the column, then sums the total score for each row within its column.
- **convert_to_dataframe:** This transformer simply converts an array to a DataFrame.


In [68]:
class TextPreprocessor(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        out = X.apply(lambda x: self._preprocess_text(x))
        return pd.DataFrame(out)
    
    def _preprocess_text(self, text):
        """ 
        Removes numbers and punvtuation marks
        """
        if text is None:
            return "No name register"
        text = re.sub(r'\d+', '', text)
        text = re.sub(r'[^\w\s]+', ' ', text)
        text = text.lower()
        return text if text.strip() else "just number"
    
class TfidfSumVectorizer(BaseEstimator, TransformerMixin):

    def __init__(self):
        self.vectorizer = TfidfVectorizer()
        
    def fit(self, X, y=None):
        self.vectorizer.fit(X)
        return self

    def transform(self, X):
        """
        Use TfidfVectorizer to assigs score to each word and the sums 
        the score of the whole sentence
        """
        tfidf_matrix = self.vectorizer.transform(X)
        row_sum = np.sum(csr_matrix(tfidf_matrix).todense(), axis=1)
        return row_sum
    

def convert_to_dataframe(X):
    """
    to convert to pd Data Frame
    """
    X = pd.DataFrame(X, columns=c)
    return X

Next, I define the transformers and specify the columns to which they are applied. Then, I create a pipeline where the entire workflow is defined. Finally, I use MinMaxScaler to scale the data.

In [69]:


text_transformer = ColumnTransformer(
    transformers=[
        ('title', TextPreprocessor(), 'title'),
    ],
    remainder='passthrough'  
)

column_transformer = ColumnTransformer(
    transformers=[
        ('tfidf_title', TfidfSumVectorizer(), 'title'),
    ],
    remainder='passthrough'  
)

pipeline = Pipeline([
    ('text', text_transformer),
    ('conversion', FunctionTransformer(func=convert_to_dataframe, validate=False)),
    ('tfidf', column_transformer)
])

X_2 = pipeline.fit_transform(X)
X_2 = pd.DataFrame(X_2, columns=X.columns)

min_max_scaler = MinMaxScaler(feature_range=(0,1))

X_2 = min_max_scaler.fit_transform(X_2)


Then, i use nearest neighbors and fit the algoritm to the data.

In [70]:
model_knn = NearestNeighbors(metric='minkowski', algorithm='brute')
model_knn.fit(X_2)

NearestNeighbors(algorithm='brute')

Finally, i use the user recommended games to recommend him 3 games for each game he recommends.

In [71]:
for game in user["title"]:
    indice_fila = games_df.loc[games_df['title'] == game].index[0]
    genres = games_df.loc[games_df['title'] == game]["genres"]
    datos_game = X_2[indice_fila]
    datos_game = pd.DataFrame(datos_game).T
    
    print(f'Game: {game}')
    print(f'Genres: {genres.values[0]}')
    
    distances, indices = model_knn.kneighbors(datos_game, n_neighbors=4)
    recommended_users = [games_df.iloc[i]['title'] for i in indices.flatten() if games_df.iloc[i]['title'] != game]
    print(f'Recommended games: {recommended_users}\n')
    

Game: Papers, Please
Genres: Adventure;Indie
Recommended games: ['Asemblance: Oversight', 'Neptune Flux', 'Castle Agony']

Game: Microsoft Flight Simulator X: Steam Edition
Genres: Simulation
Recommended games: ['X-Plane 11', 'Take Off - The Flight Simulator', 'Wings Over Europe']



## Conclusion

- The algorithm successfully recommends games that align with user preferences.
- The games recommened has a lot in common with the game recommended by the user.